<a href="https://colab.research.google.com/github/JoanChege/CHATBOT1/blob/main/chatbottrial2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.DATA PREPARATION

In [6]:
#first we need to import the required packages
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [12]:
#we load the json file and extract the required data.
with open('/content/dataset.json') as file:
    data = json.load(file)

training_sentences = []
#The variable “training_sentences” holds all the training data
#(which are the sample messages in each intent category)
training_labels = []
#the “training_labels” variable holds all the target labels correspond to each training data.
labels = []
responses = []
ignoreLetters = ['?', '!', '.', ',']


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

num_classes = len(labels)

Then we use “LabelEncoder()” function provided by scikit-learn
to convert the target labels into a form the model can understand

In [13]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

2.DATA PREPROCESSING

We vectorize our text data corpus by using the “Tokenizer” class and it allows us to limit our vocabulary size up to some defined number.
By default all punctuations will be removed, turning the texts into space-separated sequences of words, and these sequences are then split into lists of tokens. They will then be indexed or vectorized.
We can also add “oov_token” which is a value for “out of token” to deal with out of vocabulary words(tokens) at inference time.
The “pad_sequences” method is used to make all the training text sequences into the same size.

In [14]:
#Tokenize the data and perform data cleaning
vocab_size = 1000 #this is the number of words used to train the model
embedding_dim = 16 #specifies the number of dimensions that each categorical variable will be converted into.
max_len = 20 #this is the maximum number of token the model can handle
oov_token = "<OOV>" #this will replace any unknown word with a token of our choosing

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)


In [15]:
print(data)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'Hey', 'Is anyone there?', 'Hi there', 'Hello', 'Hey there', 'Howdy', 'Hola', 'Bonjour', 'Konnichiwa', 'Guten tag', 'Ola'], 'responses': ['Hello there. Tell me how are you feeling today?', 'Hi there. What brings you here today?', 'Hi there. How are you feeling today?', 'Great to see you. How do you feel currently?', "Hello there. Glad to see you're back. What's going on in your world right now?"]}, {'tag': 'morning', 'patterns': ['Good morning'], 'responses': ["Good morning. I hope you had a good night's sleep. How are you feeling today? "]}, {'tag': 'afternoon', 'patterns': ['Good afternoon'], 'responses': ['Good afternoon. How is your day going?']}, {'tag': 'evening', 'patterns': ['Good evening'], 'responses': ['Good evening. How has your day been?']}, {'tag': 'night', 'patterns': ['Good night'], 'responses': ['Good night. Get some proper sleep', 'Good night. Sweet dreams.']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 

In [16]:
#set json data into a pandas dataframe and display
#df = pd.json_normalize(data)
import pandas as pd

df = pd.json_normalize(data, record_path=['intents'])
df

,tag,patterns,responses
0,greeting,"[Hi, Hey, Is anyone there?, Hi there, Hello, H...",[Hello there. Tell me how are you feeling toda...
1,morning,[Good morning],[Good morning. I hope you had a good night's s...
2,afternoon,[Good afternoon],[Good afternoon. How is your day going?]
3,evening,[Good evening],[Good evening. How has your day been?]
4,night,[Good night],"[Good night. Get some proper sleep, Good night..."
...,...,...,...
86,self-esteem,"[I don't feel good about myself, I'm my own wo...",[You deserve to feel good about yourself. Let'...
87,trauma,"[I've experienced trauma, I can't stop thinkin...",[Trauma can be really tough to deal with. It's...
88,grief,"[I'm grieving the loss of a loved one, I don't...",[Grief is a natural and important process. It'...
89,sick,"[I feel unwell, I feel sick, I have been feeli...",[I'm sorry to hear that. Have you seen a docto...


3.MODEL TRAINING

In [10]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 91)                1547      
                                                                 
Total params: 18091 (70.67 KB)
Trainable params: 18091 (70.67 KB)
Non-trainable params: 0 (0.00 Byte)
____________________

In [11]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)


Epoch 1/500
9/9 [==============================] - 1s 4ms/step - loss: 4.5111 - accuracy: 0.0036
Epoch 2/500
9/9 [==============================] - 0s 5ms/step - loss: 4.5069 - accuracy: 0.0327
Epoch 3/500
9/9 [==============================] - 0s 3ms/step - loss: 4.5035 - accuracy: 0.0255
Epoch 4/500
9/9 [==============================] - 0s 3ms/step - loss: 4.4993 - accuracy: 0.0582
Epoch 5/500
9/9 [==============================] - 0s 3ms/step - loss: 4.4947 - accuracy: 0.0400
Epoch 6/500
9/9 [==============================] - 0s 4ms/step - loss: 4.4892 - accuracy: 0.0400
Epoch 7/500
9/9 [==============================] - 0s 4ms/step - loss: 4.4826 - accuracy: 0.0582
Epoch 8/500
9/9 [==============================] - 0s 4ms/step - loss: 4.4740 - accuracy: 0.0436
Epoch 9/500
9/9 [==============================] - 0s 4ms/step - loss: 4.4617 - accuracy: 0.0400
Epoch 10/500
9/9 [==============================] - 0s 4ms/step - loss: 4.4458 - accuracy: 0.0400
Epoch 11/500
9/9 [===========